# This tutorial shows how to use following features:

- pretty print feature for metadata.
- Excel stats for bytes and hash.
- **to_markdown** feature for metadata.
- **to_excel_template** feature to export table schema as excel.

Install frictionless library

In [1]:
! pip install frictionless[html,pandas]

In [2]:
from frictionless import Schema, Package, Resource, transform, steps
from frictionless.plugins.html import HtmlDialect
import os
from openpyxl import load_workbook

To check the latest version for frictionless

In [3]:
! frictionless --version

4.32.0


# Get Data: Land usage by country

We will use the landuse data by country: https://en.wikipedia.org/wiki/Land_use_statistics_by_country . The following code parses the html data using '**.wikitable.sortable**' selector and returns the resource object. 

to_view() property of resource object displays the data in table view.

In [4]:
fertilizer_producers = Resource(
    "https://en.wikipedia.org/wiki/Fertiliser_use_in_Nepal",
    name="fertilizer_producers",
    format="html",
    dialect=HtmlDialect(selector=".wikitable.sortable"),
)
print(fertilizer_producers.to_view())

+--------------------------------------------------------------+--------------------+
| Name                                                         | Capacity (MT/year) |
+==============================================================+====================+
| 'Unique Bio Tech Organic Pvt. Ltd, Chitwan'                  | [20.0, 0.0]        |
+--------------------------------------------------------------+--------------------+
| 'Buddha Organic fertiliser Industry, Dhanusa'                | [10.0, 0.0]        |
+--------------------------------------------------------------+--------------------+
| 'Janakpur fertilisers Industries, Morang'                    | [10.0, 0.0]        |
+--------------------------------------------------------------+--------------------+
| 'Annapurna Agriculture fertiliser Industry, Morang'          | [10.0, 0.0]        |
+--------------------------------------------------------------+--------------------+
| 'Nepal Integrated Model Agro Firm Pvt. Ltd, Kavrepal

**Transform to package**

We will use the transform feature of frictionless to further describe our metadata using additional properties: '**type**' and '**description**' . 

In [5]:
fertilizer_producers = transform(
    fertilizer_producers,
    steps=[
      steps.field_update(name="Name", type="string", description="Fertilizer Producer"),
      steps.field_update(name="Capacity (MT/year)", type="string", description="Production Capacity"),
    ]
)

Here we package our resource into a container named "farmers" which is a frictionless Package class and gave a description to it. They are metadata for package. 

In this example, we have only one resource but Package can be used to combine multiple resources. 

In [6]:
farmers = Package(name="farmers", description="information about fertilizer producer", resources=[fertilizer_producers])

# Pretty print metadata

Now you do not have to use pprint to format the metadata!. All the metadata are automatically converted to pretty print format which displays the metadata in readable and pretty way.

In [7]:
# Resource
fertilizer_producers

{'dialect': {'selector': '.wikitable.sortable'},
 'encoding': 'utf-8',
 'format': 'html',
 'hashing': 'md5',
 'name': 'fertilizer_producers',
 'path': 'https://en.wikipedia.org/wiki/Fertiliser_use_in_Nepal',
 'profile': 'tabular-data-resource',
 'schema': {'fields': [{'description': 'Fertilizer Producer',
                        'name': 'Name',
                        'type': 'string'},
                       {'description': 'Production Capacity',
                        'name': 'Capacity (MT/year)',
                        'type': 'string'}]},
 'scheme': 'https'}

In [8]:
# Schema
fertilizer_producers.schema

{'fields': [{'description': 'Fertilizer Producer',
             'name': 'Name',
             'type': 'string'},
            {'description': 'Production Capacity',
             'name': 'Capacity (MT/year)',
             'type': 'string'}]}

# Get Markdown

Metadata now supports conversion to markdown format using the new feature '**to_markdown**.'

In [9]:
# Package
print(farmers.to_markdown())

# `farmers`- `description` information about fertilizer producer
## `fertilizer_producers`
  - `path` https://en.wikipedia.org/wiki/Fertiliser_use_in_Nepal
  - `schema`
      
### `Name`
  - `description` Fertilizer Producer
  - `type` string
### `Capacity (MT/year)`
  - `description` Production Capacity
  - `type` string


In [10]:
# Resource
print(fertilizer_producers.to_markdown())

## `fertilizer_producers`
  - `path` https://en.wikipedia.org/wiki/Fertiliser_use_in_Nepal
  - `schema`
      
### `Name`
  - `description` Fertilizer Producer
  - `type` string
### `Capacity (MT/year)`
  - `description` Production Capacity
  - `type` string


In [11]:
# Schema
print(fertilizer_producers.schema.to_markdown())

## `schema`

### `Name`
  - `description` Fertilizer Producer
  - `type` string
### `Capacity (MT/year)`
  - `description` Production Capacity
  - `type` string


# Export table to excel

The schema metadata can now be exported to excel using function '**to_excel_template**'

In [12]:
fertilizer_producers.schema.to_excel_template('fertilizer_producers.xlsx')

# Excel Stats

In excel stats we can now see additional information about excel: "hash" and "bytes". Lets open an excel sheet that we created in above step using **to_excel_template**. 

And add few data to it.

In [13]:
# open the workbook and add some data
fertilizer_producers_wb = load_workbook('fertilizer_producers.xlsx')
page = fertilizer_producers_wb.active

# New data to write:
new_producers = [['abc','20,000'], ['def','30000']]

for info in new_producers:
    page.append(info)

fertilizer_producers_wb.save(filename='fertilizer_producers.xlsx')

Now we can then use '**.infer**' and '**.stats**' property to view information of the excel file. Stats now includes hash and bytes data of excel sheet as shown in the example

In [14]:
fertilizer_producers_rs = Resource("fertilizer_producers.xlsx")
fertilizer_producers_rs.infer(stats=True)
print(fertilizer_producers_rs.stats)

{'hash': '1d3f7cfb0efa4c0bfc7f702739ded4a5', 'bytes': 6307, 'fields': 2, 'rows': 2}
